In [9]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/ximo/Documents/GitHub/skforecast')
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput

The importance of predictors can be obtained using the methods `get_coef` and `get_feature_importance`. These methods access the attributes `coef_` and `feature_importances_` of the internal regressor, respectively.

> **⚠ WARNING:**  
> This methods only return values if the regressor used inside the forecaster has the attribute `coef_` or `feature_importances_`.


In [11]:
# Download data
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o_exog.csv')
data = pd.read_csv(url, sep=',', header=0, names=['date', 'y', 'exog_1', 'exog_2'])

# Data preprocessing
# ==============================================================================
data['date'] = pd.to_datetime(data['date'], format='%Y/%m/%d')
data = data.set_index('date')
data = data.asfreq('MS')

In [12]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(),
                lags = 5
             )

forecaster.fit(y=data['y'], exog=data[['exog_1', 'exog_2']])

# Predictors importance
# ==============================================================================
print(forecaster.get_feature_importance().to_markdown(tablefmt="github"))

|    | feature   |   importance |
|----|-----------|--------------|
|  0 | lag_1     |    0.558158  |
|  1 | lag_2     |    0.106364  |
|  2 | lag_3     |    0.0226901 |
|  3 | lag_4     |    0.0545216 |
|  4 | lag_5     |    0.0652243 |
|  5 | exog_1    |    0.0462155 |
|  6 | exog_2    |    0.146827  |


In [13]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                    regressor = Ridge(),
                    lags = 5
                )

forecaster.fit(y=data['y'], exog=data[['exog_1', 'exog_2']])


print(forecaster.get_coef().to_markdown(tablefmt="github"))

|    | feature   |       coef |
|----|-----------|------------|
|  0 | lag_1     |  0.327688  |
|  1 | lag_2     | -0.0735932 |
|  2 | lag_3     | -0.152202  |
|  3 | lag_4     | -0.217106  |
|  4 | lag_5     | -0.1458    |
|  5 | exog_1    |  0.379798  |
|  6 | exog_2    |  0.668162  |


When using a `ForecasterAutoregMultiOutput`, since a different model is fit for each step, it is needed to indicate from which model extract the information.

In [14]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterAutoregMultiOutput(
                regressor = RandomForestRegressor(),
                steps = 10,
                lags = 5
             )

forecaster.fit(y=data['y'], exog=data[['exog_1', 'exog_2']])

# Predictors importance
# ==============================================================================
print(forecaster.get_feature_importance(step=1).to_markdown(tablefmt="github"))

|    | feature   |   importance |
|----|-----------|--------------|
|  0 | lag_1     |    0.529107  |
|  1 | lag_2     |    0.109431  |
|  2 | lag_3     |    0.0197598 |
|  3 | lag_4     |    0.0813373 |
|  4 | lag_5     |    0.0480799 |
|  5 | exog_1    |    0.0371122 |
|  6 | exog_2    |    0.175173  |
